Questo notebook integra i dati di ANAC con quelli di IndicePA e contiene una serie di analisi descrittive

**Note**
1. il numero di righe del file in input (i.e. 4819304) non corrisponde alla dimensione del dataset (i.e. 4771894). Non sembra esserci una ragione.

In [1]:
import pandas as pd
import numpy
import csv

In [2]:
f_anac = "data/anac/anacDataset_1522836870094_postprocessed2.tsv"
#df = pd.read_csv(f_anac, sep="\t", error_bad_lines=False, parse_dates=True)
df = pd.read_csv(f_anac, sep="\t", parse_dates=True)

/Users/fabiana/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#ci sono typo in alcune date (e.g. 202/02/2016)
df['dataInizio'] = pd.to_datetime(df['dataInizio'], errors='coerce', format="%d/%m/%Y")
df['dataUltimazione'] = pd.to_datetime(df['dataUltimazione'], errors='coerce', format="%d/%m/%Y")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3913440 entries, 0 to 3913439
Data columns (total 17 columns):
cig                                  object
cfStrutturaProponente                object
denominazioneStrutturaProponente     object
oggetto                              object
sceltaContraente                     object
importoAggiudicazione                float64
importoSommeLiquidate                float64
dataInizio                           datetime64[ns]
dataUltimazione                      datetime64[ns]
jsonPartecipanti                     object
jsonAggiudicatari                    object
totalePartecipanti                   int64
totaleAggiudicatari                  int64
totaleRaggruppamentopartecipanti     int64
totaleRaggruppamentoAggiudicatari    int64
cfPrimoaggiudicatario                object
denominazionePrimoaggiudicatario     object
dtypes: datetime64[ns](2), float64(2), int64(4), object(9)
memory usage: 507.6+ MB


In [5]:
df.head(3)

,cig,cfStrutturaProponente,denominazioneStrutturaProponente,oggetto,sceltaContraente,importoAggiudicazione,importoSommeLiquidate,dataInizio,dataUltimazione,jsonPartecipanti,jsonAggiudicatari,totalePartecipanti,totaleAggiudicatari,totaleRaggruppamentopartecipanti,totaleRaggruppamentoAggiudicatari,cfPrimoaggiudicatario,denominazionePrimoaggiudicatario
0,0000000000,7640210964,AMCeM Azienda Multiservizi Cerro Maggiore,Attrezzature per autoanalisi,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,3306.00,0.00,2016-11-25,2016-11-25,"{""raggruppamento"":[],""partecipante"":[{""codiceF...","{""aggiudicatarioRaggruppamento"":[],""aggiudicat...",3,1,0,0,01615440342,Callegari Srl
1,0000000000,7640210964,AMCeM Azienda Multiservizi Cerro Maggiore,Cancelleria,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,2307.34,2307.34,2012-03-31,NaT,"{""raggruppamento"":[],""partecipante"":[{""codiceF...","{""aggiudicatarioRaggruppamento"":[],""aggiudicat...",1,1,0,0,06450560963,CV office SAS di Del Freo Chiara E C
2,0000000000,7640210964,AMCeM Azienda Multiservizi Cerro Maggiore,Telefonia,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,4160.93,4160.93,2015-02-25,NaT,"{""raggruppamento"":[],""partecipante"":[{""codiceF...","{""aggiudicatarioRaggruppamento"":[],""aggiudicat...",3,1,0,0,08539010010,Vodafone Italia SpA


In [6]:
is_2017 = (df['dataInizio'] > '01/01/2017') & (df['dataInizio'] <= '31/12/2017')
df_2017 = df.loc[is_2017]
df_2017.shape

(1842313, 17)

In [8]:
df_cfPrimoaggiudicatario_count = df_2017[["cfStrutturaProponente", "cfPrimoaggiudicatario"]].groupby(["cfPrimoaggiudicatario"])["cfPrimoaggiudicatario"].agg(["count"]).reset_index(0)#.sort_values(['count'], ascending=False)
df_cfPrimoaggiudicatario_count.head(3)

,cfPrimoaggiudicatario,count
0,,146
1,,4
2,,1


In [9]:
df_cfPrimoaggiudicatario_sum = df_2017[["importoSommeLiquidate","importoAggiudicazione", "cfPrimoaggiudicatario"]].groupby(["cfPrimoaggiudicatario"]).sum().reset_index()
df_cfPrimoaggiudicatario_sum.head(3)

,cfPrimoaggiudicatario,importoSommeLiquidate,importoAggiudicazione
0,,331158.77,331370.77
1,,4066.00,4066.00
2,,744.66,744.66


In [10]:
print("df_cfPrimoaggiudicatario_count", df_cfPrimoaggiudicatario_count.shape)
print("df_cfPrimoaggiudicatario_sum", df_cfPrimoaggiudicatario_sum.shape)

df_cfPrimoaggiudicatario_count (315276, 2)
df_cfPrimoaggiudicatario_sum (315276, 3)


In [11]:
#df_cfPrimoaggiudicatario_sum.sort_values(["importoSommeLiquidate"], ascending=False).head()

In [12]:
df_denominazione_aggiudicatario = df_2017[["cfPrimoaggiudicatario","denominazionePrimoaggiudicatario"]]
df_denominazione_aggiudicatario = df_denominazione_aggiudicatario.set_index("cfPrimoaggiudicatario")
df_denominazione_aggiudicatario = df_denominazione_aggiudicatario[~df_denominazione_aggiudicatario.index.duplicated(keep='first')]

df_denominazione_aggiudicatario.head(3)

,denominazionePrimoaggiudicatario
cfPrimoaggiudicatario,
09606050962,Kresco Group Multibrand srl
Raggruppamento,Raggruppamento
01640430243,LIBRERIA PALAZZO ROBERTI


In [13]:
df_join = df_cfPrimoaggiudicatario_count.set_index('cfPrimoaggiudicatario').join(df_cfPrimoaggiudicatario_sum.set_index('cfPrimoaggiudicatario'))
df_join.head(3)

,count,importoSommeLiquidate,importoAggiudicazione
cfPrimoaggiudicatario,,,
,146,331158.77,331370.77
,4,4066.00,4066.00
,1,744.66,744.66


In [14]:
df_join.shape

(315276, 3)

In [15]:
#Elimino i duplicati dalle denominazioni
df_denominazione_aggiudicatario = df_denominazione_aggiudicatario[~df_denominazione_aggiudicatario.index.duplicated(keep='first')]
df_denominazione_aggiudicatario.shape

(315277, 1)

In [16]:
final_df = pd.merge(df_denominazione_aggiudicatario, df_join, left_index=True, right_index=True, how='inner')
final_df.shape

(315276, 4)

In [17]:
final_df.sort_values(['count', "importoAggiudicazione"], ascending=False).head()

,denominazionePrimoaggiudicatario,count,importoSommeLiquidate,importoAggiudicazione
cfPrimoaggiudicatario,,,,
Raggruppamento,Raggruppamento,74668,1.290753e+09,5.298415e+09
00150470342,Gruppo Spaggiari Parma SpA,14857,7.373984e+06,8.166934e+06
00488410010,TELECOM ITALIA SpA,10502,3.060910e+07,1.156872e+08
02027040019,CDATABORGIONE CENTRO DIDATTICO SRL,7522,3.413943e+06,3.970015e+06
03222970406,MyO Srl,7002,3.963308e+06,7.912744e+06


In [18]:
df_2017_00150470342 = df_2017.loc[df_2017["cfPrimoaggiudicatario"]=="00150470342"]

In [21]:
df_2017_00150470342.head()

,cig,cfStrutturaProponente,denominazioneStrutturaProponente,oggetto,sceltaContraente,importoAggiudicazione,importoSommeLiquidate,dataInizio,dataUltimazione,jsonPartecipanti,jsonAggiudicatari,totalePartecipanti,totaleAggiudicatari,totaleRaggruppamentopartecipanti,totaleRaggruppamentoAggiudicatari,cfPrimoaggiudicatario,denominazionePrimoaggiudicatario
3657,Z0F1CE43C2,90000850827,ISTITUTO COMPRENSIVO STATALE IGNAZIO BUTTITTA...,libretti giustificaz assenze fattura n 20174E...,04PROCEDURA NEGOZIATA SENZA PREVIA PUBBLICAZIO...,170.0,207.4,2017-02-09,NaT,"{""raggruppamento"":[],""partecipante"":[{""codiceF...","{""aggiudicatarioRaggruppamento"":[],""aggiudicat...",1,1,0,0,00150470342,Gruppo Spaggiari Parma SpA
3674,Z6A1E87438,90000850827,ISTITUTO COMPRENSIVO STATALE IGNAZIO BUTTITTA...,Imponibile su materiale per esami fattura n 2...,04PROCEDURA NEGOZIATA SENZA PREVIA PUBBLICAZIO...,283.2,345.5,2017-05-25,NaT,"{""raggruppamento"":[],""partecipante"":[{""codiceF...","{""aggiudicatarioRaggruppamento"":[],""aggiudicat...",1,1,0,0,00150470342,Gruppo Spaggiari Parma SpA
3754,Z881E7E25F,80004030765,ISTITUTO COMPRENSIVO STATALE A BUSCIOLANO PO...,Pagamento Fattura n20174E15596 del 09052017,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,48.5,48.5,2017-05-23,2017-05-23,"{""raggruppamento"":[],""partecipante"":[{""codiceF...","{""aggiudicatarioRaggruppamento"":[],""aggiudicat...",1,1,0,0,00150470342,SPAGGIARI SPA
3769,ZA61FE5A08,80004030765,ISTITUTO COMPRENSIVO STATALE A BUSCIOLANO PO...,Pagamento Fattura n20174E26915 del 15092017,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,559.2,559.2,2017-10-04,2017-10-04,"{""raggruppamento"":[],""partecipante"":[{""codiceF...","{""aggiudicatarioRaggruppamento"":[],""aggiudicat...",1,1,0,0,00150470342,SPAGGIARI SPA
3784,ZBF1D77A29,80004030765,ISTITUTO COMPRENSIVO STATALE A BUSCIOLANO PO...,Pagamento Fattura n20174E06376 del 27022017,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,537.6,537.6,2017-03-24,2017-03-24,"{""raggruppamento"":[],""partecipante"":[{""codiceF...","{""aggiudicatarioRaggruppamento"":[],""aggiudicat...",1,1,0,0,00150470342,SPAGGIARI SPA


In [20]:
df_2017_00150470342_count = df_2017_00150470342.groupby(["cfStrutturaProponente"])["cfStrutturaProponente"].agg(["count"]).reset_index(0)#.sort_values(['count'], ascending=False)

df_2017_00150470342_sum = df_2017_00150470342[["importoSommeLiquidate","importoAggiudicazione", "cfStrutturaProponente"]].groupby(["cfStrutturaProponente"]).sum().reset_index()
print(df_2017_00150470342_sum.head(3))
print(df_2017_00150470342_count.head(3))

  cfStrutturaProponente  importoSommeLiquidate  importoAggiudicazione
0                     0                1410.45                1808.17
1              69850949                   0.00                  92.80
2             101350445                1000.00                1000.00
  cfStrutturaProponente  count
0                     0      2
1              69850949      1
2             101350445      1


In [56]:
df_2017_00150470342.loc[df_2017_00150470342["cfStrutturaProponente"]==0]

,cig,cfStrutturaProponente,denominazioneStrutturaProponente,oggetto,sceltaContraente,importoAggiudicazione,importoSommeLiquidate,dataInizio,dataUltimazione,jsonPartecipanti,jsonAggiudicatari,totalePartecipanti,totaleAggiudicatari,totaleRaggruppamentopartecipanti,totaleRaggruppamentoAggiudicatari,cfPrimoaggiudicatario,denominazionePrimoaggiudicatario
98285,ZAF1E5A195,0,COMUNE DI CHIAVARI,FORNITURA REGISTRI PER ISTITUTO COMPRENSIV...,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,397.72,0.00,2017-04-18,NaT,"{""raggruppamento"":[],""partecipante"":[{""codiceF...","{""aggiudicatarioRaggruppamento"":[],""aggiudicat...",1,1,0,0,00150470342,CASA EDITRICE SPAGGIARI SPA
98630,ZEE1FED757,0,COMUNE DI CHIAVARI,FORNITURA REGISTRI PER ISTITUTO COMPRENSIV...,23AFFIDAMENTO IN ECONOMIA AFFIDAMENTO DIRETTO,1410.45,1410.45,2017-09-18,2017-12-31,"{""raggruppamento"":[],""partecipante"":[{""codiceF...","{""aggiudicatarioRaggruppamento"":[],""aggiudicat...",1,1,0,0,00150470342,CASA EDITRICE SPAGGIARI SPA


In [58]:
df_2017_00150470342.shape

(14857, 17)

In [60]:
df_2017_00150470342_count2 = df_2017_00150470342.groupby(["denominazioneStrutturaProponente"])["denominazioneStrutturaProponente"].agg(["count"]).reset_index(0)#.sort_values(['count'], ascending=False)

df_2017_00150470342_sum2 = df_2017_00150470342[["importoSommeLiquidate","importoAggiudicazione", "denominazioneStrutturaProponente"]].groupby(["denominazioneStrutturaProponente"]).sum().reset_index()
print(df_2017_00150470342_sum.head(3))
print(df_2017_00150470342_count.head(3))


                    denominazioneStrutturaProponente  importoSommeLiquidate  \
0    ISTITUTO COMPRENSIVO DI ROVEREDO IN PIANO GI...                 1875.0   
1                           Liceo Scientifico A DIAZ                  468.0   
2                                  MAGISTRI CUMACINI                  705.0   

   importoAggiudicazione  
0                 1875.0  
1                  468.0  
2                  705.0  
                    denominazioneStrutturaProponente  count
0    ISTITUTO COMPRENSIVO DI ROVEREDO IN PIANO GI...      6
1                           Liceo Scientifico A DIAZ      1
2                                  MAGISTRI CUMACINI      6


In [62]:
print(df_2017_00150470342_count2.shape)
print(df_2017_00150470342_count.shape)

(3535, 2)
(3536, 2)


In [63]:
df_2017_00150470342_count2

,denominazioneStrutturaProponente,count
0,ISTITUTO COMPRENSIVO DI ROVEREDO IN PIANO GI...,6
1,Liceo Scientifico A DIAZ,1
2,MAGISTRI CUMACINI,6
3,CDATAISIS Carlo Levi,2
4,CDATAISTITUTO COMPRENSIVO CLAUDIO PUDDU PRATO,2
5,CDATAISTITUTO COMPRENSIVO LENTINI,1
6,CDATAISTITUTO COMPRENSIVO MARCO POLO,10
7,CDATAIstituto Istruzione Superiore Miraglia,4
8,CDATAIstituto Superiore Corridoni Campana,14
9,CPIA MILANO,3
